In [1]:
import pandas as pd
import numpy as np
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from numpy import linalg as LA
import warnings
warnings.filterwarnings('ignore')

In [2]:
genscore = pd.read_csv('gen.txt')

In [3]:
genscore

,movieId,tagId,relevance
0,1,1,0.02500
1,1,2,0.02500
2,1,3,0.05775
3,1,4,0.09675
4,1,5,0.14675
...,...,...,...
1872475,65514,1124,0.03400
1872476,65514,1125,0.98800
1872477,65514,1126,0.11375
1872478,65514,1127,0.13675


In [4]:
tags = pd.read_csv('tags.txt')

In [5]:
tags

,tagId,tag
0,1,007
1,2,007 (series)
2,3,18th century
3,4,1920s
4,5,1930s
...,...,...
1123,1124,writing
1124,1125,wuxia
1125,1126,wwii
1126,1127,zombie


In [6]:
items = [i.strip('\n').split('::') for i in open('items.txt')]

In [7]:
items = pd.DataFrame(items)

In [8]:
items[1] = items[1].astype('str')

In [9]:
items[0] = items[0].astype('int')

In [10]:
items[items[0] == 65514]

,0,1,2
1562,65514,Ip Man (2008),Action|Drama|War


In [11]:
k = tags['tagId'].tolist()
v = tags['tag'].tolist()
k1 = items[0].tolist()
v1 = items[1].tolist()

dic = dict(zip(k, v))
dic1 = dict(zip(k1, v1))
genscore['tag'] = genscore['tagId'].map(dic)
genscore['name'] = genscore['movieId'].map(dic1)

In [12]:
genscore

,movieId,tagId,relevance,tag,name
0,1,1,0.02500,007,Toy Story (1995)
1,1,2,0.02500,007 (series),Toy Story (1995)
2,1,3,0.05775,18th century,Toy Story (1995)
3,1,4,0.09675,1920s,Toy Story (1995)
4,1,5,0.14675,1930s,Toy Story (1995)
...,...,...,...,...,...
1872475,65514,1124,0.03400,writing,Ip Man (2008)
1872476,65514,1125,0.98800,wuxia,Ip Man (2008)
1872477,65514,1126,0.11375,wwii,Ip Man (2008)
1872478,65514,1127,0.13675,zombie,Ip Man (2008)


In [13]:
genscore = genscore.sort_values(by=["relevance"], ascending=False)

In [14]:
genscore = genscore[['movieId', 'name', 'tagId', 'tag', 'relevance']]

In [15]:
genscore

,movieId,name,tagId,tag,relevance
1810801,30749,Hotel Rwanda (2004),362,ethnic conflict,1.00000
1559164,55805,Before the Devil Knows You're Dead (2007),269,crime gone awry,1.00000
1105441,3639,"""Man with the Golden Gun, The (1974)""",2,007 (series),1.00000
1869111,48982,Flushed Away (2006),16,aardman studios,1.00000
1103185,3633,On Her Majesty's Secret Service (1969),2,007 (series),1.00000
...,...,...,...,...,...
798497,2424,You've Got Mail (1998),1002,swedish,0.00075
799970,2432,Stepmom (1998),219,claymation,0.00075
261914,748,"""Arrival, The (1996)""",219,claymation,0.00075
1352345,4823,Serendipity (2001),1002,swedish,0.00075


In [16]:
grouped_df = genscore.groupby("movieId")
genscore = grouped_df.head(100)
genscore = genscore.reset_index()
print(genscore)

          index  movieId                                       name  tagId  \
0       1810801    30749                        Hotel Rwanda (2004)    362   
1       1559164    55805  Before the Devil Knows You're Dead (2007)    269   
2       1105441     3639      "Man with the Golden Gun, The (1974)"      2   
3       1869111    48982                        Flushed Away (2006)     16   
4       1103185     3633     On Her Majesty's Secret Service (1969)      2   
...         ...      ...                                        ...    ...   
165995  1831587    31433                 "Wedding Date, The (2005)"    844   
165996   325500     1006                      "Chamber, The (1996)"    637   
165997   325954     1006                      "Chamber, The (1996)"   1091   
165998  1831222    31433                 "Wedding Date, The (2005)"    479   
165999  1831617    31433                 "Wedding Date, The (2005)"    874   

                       tag  relevance  
0          ethnic confl

In [17]:
genscore = pd.DataFrame(genscore)

In [18]:
genscore =genscore.sort_values(by=["movieId"], ascending=True)

In [19]:
genscore[genscore['movieId'] == 1].sort_values(by=["relevance"], ascending=False)

,index,movieId,name,tagId,tag,relevance
150,1035,1,Toy Story (1995),1036,toys,0.99925
285,243,1,Toy Story (1995),244,computer animation,0.99850
614,785,1,Toy Story (1995),786,pixar animation,0.99600
1347,588,1,Toy Story (1995),589,kids and family,0.99075
1986,63,1,Toy Story (1995),64,animation,0.98575
...,...,...,...,...,...,...
99366,366,1,Toy Story (1995),367,exciting,0.47075
100654,452,1,Toy Story (1995),453,good story,0.46675
101156,941,1,Toy Story (1995),942,space,0.46475
101777,377,1,Toy Story (1995),378,fantasy world,0.46300


In [20]:
genscore

,index,movieId,name,tagId,tag,relevance
21179,214,1,Toy Story (1995),215,classic,0.81925
41429,416,1,Toy Story (1995),417,funny,0.69500
34304,451,1,Toy Story (1995),452,good soundtrack,0.73375
19022,444,1,Toy Story (1995),445,good,0.83550
49178,829,1,Toy Story (1995),830,quotable,0.65725
...,...,...,...,...,...,...
28711,686822,116797,The Imitation Game (2014),999,suspense,0.76750
50778,686885,116797,The Imitation Game (2014),1062,unlikely friendships,0.65000
6462,686459,116797,The Imitation Game (2014),636,mathematics,0.94675
51410,686408,116797,The Imitation Game (2014),585,justice,0.64725


In [21]:
genscore = genscore[genscore['relevance'] >= 0.5].groupby(['movieId','name'])['tag'].apply(lambda x: ','.join(x)).reset_index()
genscore['tag_list'] = genscore.tag.map(lambda x: x.split(','))

In [22]:
genscore

,movieId,name,tag,tag_list
0,1,Toy Story (1995),"classic,funny,good soundtrack,good,quotable,si...","[classic, funny, good soundtrack, good, quotab..."
1,2,Jumanji (1995),"jungle,comedy,fantasy world,based on a video g...","[jungle, comedy, fantasy world, based on a vid..."
2,3,Grumpier Old Men (1995),"good sequel,chase,fish,romance,pg-13,great,sil...","[good sequel, chase, fish, romance, pg-13, gre..."
3,10,GoldenEye (1995),"plot,cool,assassin,runaway,adventure,fun movie...","[plot, cool, assassin, runaway, adventure, fun..."
4,11,"""American President, The (1995)""","happy ending,sweet,romantic,great,pg-13,love s...","[happy ending, sweet, romantic, great, pg-13, ..."
...,...,...,...,...
1655,106782,"""Wolf of Wall Street, The (2013)""","fast paced,capitalism,anti-hero,drama,golden p...","[fast paced, capitalism, anti-hero, drama, gol..."
1656,109374,"""Grand Budapest Hotel, The (2014)""","visually stunning,obsession,beautifully filmed...","[visually stunning, obsession, beautifully fil..."
1657,112556,Gone Girl (2014),"paranoia,surveillance,twist ending,bad ending,...","[paranoia, surveillance, twist ending, bad end..."
1658,112852,Guardians of the Galaxy (2014),"fight scenes,great,witty,entertaining,intense,...","[fight scenes, great, witty, entertaining, int..."


In [23]:
target_movie = 1

target_tag_list = genscore[genscore['movieId'] == target_movie].tag_list.values[0]
mv_tags_list_sim = genscore[['movieId','name','tag_list','tag']]
mv_tags_list_sim['jaccard_sim'] = mv_tags_list_sim.tag_list.map(lambda x: len(set(x).intersection(set(target_tag_list))) / len(set(x).union(set(target_tag_list))))
print(f'Movies most similar to {target_movie} based on tags:')
text = ','.join(mv_tags_list_sim.sort_values(by = 'jaccard_sim', ascending = False).head(25)['tag'].values)
mv_tags_list_sim.sort_values(by = 'jaccard_sim', ascending = False).head(10)

Movies most similar to 1 based on tags:


,movieId,name,tag_list,tag,jaccard_sim
0,1,Toy Story (1995),"[classic, funny, good soundtrack, good, quotab...","classic,funny,good soundtrack,good,quotable,si...",1.000000
1132,4886,"""Monsters, Inc. (2001)""","[disney animated feature, disney, animation, e...","disney animated feature,disney,animation,enter...",0.697917
640,2355,"""Bug's Life, A (1998)""","[humor, light, toys, pixar animation, colourfu...","humor,light,toys,pixar animation,colourful,tal...",0.626374
1250,6377,Finding Nemo (2003),"[family, mentor, cgi, childhood, visually appe...","family,mentor,cgi,childhood,visually appealing...",0.601770
1575,68954,Up (2009),"[light, kids and family, imdb top 250, nostalg...","light,kids and family,imdb top 250,nostalgic,b...",0.516667
1600,79091,Despicable Me (2010),"[pg-13, feel-good, comedy, dialogue, sweet, an...","pg-13,feel-good,comedy,dialogue,sweet,animated...",0.484848
1595,76093,How to Train Your Dragon (2010),"[great ending, feel good movie, visuals, imdb ...","great ending,feel good movie,visuals,imdb top ...",0.479675
17,34,Babe (1995),"[light, childhood, sentimental, good, based on...","light,childhood,sentimental,good,based on a bo...",0.454545
1441,45517,Cars (2006),"[disney, humor, unlikely friendships, transfor...","disney,humor,unlikely friendships,transformati...",0.409524
1551,63859,Bolt (2008),"[kids, cute!, talking animals, animation, osca...","kids,cute!,talking animals,animation,oscar (be...",0.408602


In [24]:
def jaccard(list1, list2):
    nominator = len(list1.intersection(list2))
    denominator = len(list1.union(list2))
    jac = nominator/denominator
    return jac

In [27]:
def recommend(a, b, c):
    target_tag_list1 = []
    la = genscore[genscore['movieId'] == a].tag_list.values[0]
    lb = genscore[genscore['movieId'] == b].tag_list.values[0]
    lc = genscore[genscore['movieId'] == c].tag_list.values[0]
    target_tag_list1 = [*la, *lb, *lc]
    
    mv_tags_list_sim = genscore[['movieId','name','tag_list','tag']]
    mv_tags_list_sim['jaccard'] = mv_tags_list_sim.tag_list.map(lambda x: len(set(x).intersection(set(target_tag_list1))) / len(set(x).union(set(target_tag_list1))))
    print(f'Movies most similar to last 3 watched movies:')
    text = ','.join(mv_tags_list_sim.sort_values(by = 'jaccard', ascending = False).head(25)['tag'].values)
    df = mv_tags_list_sim.sort_values(by = 'jaccard', ascending = False).head(13)
    df = df[['movieId', 'name', 'jaccard']]
    return df.iloc[3:]

In [29]:
recommend(1,4886,2355)

Movies most similar to last 3 watched movies:


,movieId,name,jaccard
1250,6377,Finding Nemo (2003),0.626016
1595,76093,How to Train Your Dragon (2010),0.534351
1575,68954,Up (2009),0.511278
1600,79091,Despicable Me (2010),0.509091
1441,45517,Cars (2006),0.415254
365,1270,Back to the Future (1985),0.395833
127,364,"""Lion King, The (1994)""",0.394958
1551,63859,Bolt (2008),0.376147
17,34,Babe (1995),0.373832
731,2716,Ghostbusters (a.k.a. Ghost Busters) (1984),0.364286
